In [1]:
# import all nesecarry packages
import pandas as pd
import streamlit as st
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import re
import math
from bs4 import BeautifulSoup
import requests

In [2]:
# read in the data
games = pd.read_csv(r'c:\Users\Darth Piggyus\Github-Projects\Intergated-project\games.csv')

In [3]:
# get an overview of the data
print(games.head())
games.info()
games.describe()

                       Name Platform  Year_of_Release         Genre  NA_sales  \
0                Wii Sports      Wii           2006.0        Sports     41.36   
1         Super Mario Bros.      NES           1985.0      Platform     29.08   
2            Mario Kart Wii      Wii           2008.0        Racing     15.68   
3         Wii Sports Resort      Wii           2009.0        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing     11.27   

   EU_sales  JP_sales  Other_sales  Critic_Score User_Score Rating  
0     28.96      3.77         8.45          76.0          8      E  
1      3.58      6.81         0.77           NaN        NaN    NaN  
2     12.76      3.79         3.29          82.0        8.3      E  
3     10.93      3.28         2.95          80.0          8      E  
4      8.89     10.22         1.00           NaN        NaN    NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 

,Year_of_Release,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


Missing values in 'Name', Year_of_Release', 'Genre', 'Critic_score', User_Score', and 'Rating'; 'Year_of_Release' should be int type; 'User_Score' should be float type; Year of Release ranges from 1980 to 2016

In [4]:
# make all column names lowercase
games.columns = games.columns.str.lower()

In [5]:
# convert 'user_score' to float
#games['user_score'] = games['user_score'].astype(float)

In [6]:
# display the rows with missing values in the 'name' column
print(games[games['name'].isnull()])

      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    NaN      GEN           1993.0   NaN      1.78      0.53      0.00   
14244  NaN      GEN           1993.0   NaN      0.00      0.00      0.03   

       other_sales  critic_score user_score rating  
659           0.08           NaN        NaN    NaN  
14244         0.00           NaN        NaN    NaN  


In [7]:
# fill the missing values in the 'name' column
games['name'] = games['name'].fillna('Unknown')

# check to makes sure there are no empty values
print(games[games['name'].isnull()])

Empty DataFrame
Columns: [name, platform, year_of_release, genre, na_sales, eu_sales, jp_sales, other_sales, critic_score, user_score, rating]
Index: []


I'm using 'Unknown' here for the names because there's really no way of know what games these are specificlly. They also shouldn't be relavent to our data since we're looking at 2016.

In [8]:
# display the rows with missing values in the 'year_of_release' column
print(games[games['year_of_release'].isnull()])

# Check for missing values in the 'year_of_release' column
print(f"Number of missing values in the 'year_of_release' column: {games['year_of_release'].isnull().sum()}")

                                    name platform  year_of_release  \
183                      Madden NFL 2004      PS2              NaN   
377                     FIFA Soccer 2004      PS2              NaN   
456           LEGO Batman: The Videogame      Wii              NaN   
475           wwe Smackdown vs. Raw 2006      PS2              NaN   
609                       Space Invaders     2600              NaN   
...                                  ...      ...              ...   
16373  PDC World Championship Darts 2008      PSP              NaN   
16405                      Freaky Flyers       GC              NaN   
16448                          Inversion       PC              NaN   
16458        Hakuouki: Shinsengumi Kitan      PS3              NaN   
16522                       Virtua Quest       GC              NaN   

              genre  na_sales  eu_sales  jp_sales  other_sales  critic_score  \
183          Sports      4.26      0.26      0.01         0.71          94.0   

First I can check the name of each game for a year and use that to fill for those. The rest I'll fill by grouping by platform and find the median year

In [9]:
# Function to extract year from game name
def extract_year_from_name(name):
    year_pattern = r'\b(\d{4})\b'
    match = re.search(year_pattern, name)
    if match:
        return int(match.group(1))
    else:
        return None

# Apply the function to the 'name' column and update the 'year_of_release' column only for rows with missing values
games.loc[games['year_of_release'].isnull(), 'year_of_release'] = games.loc[games['year_of_release'].isnull(), 'name'].apply(extract_year_from_name)

# Check for missing values in the 'year_of_release' column
print(f"Number of missing values in the 'year_of_release' column: {games['year_of_release'].isnull().sum()}")

Number of missing values in the 'year_of_release' column: 252


In [10]:
# Group the data by 'platform' and calculate the median 'year_of_release' for each group
platform_medians = games.groupby('platform')['year_of_release'].median().round().to_dict()

# Fill the missing values in the 'year_of_release' column using the platform medians
games.loc[games['year_of_release'].isnull(), 'year_of_release'] = games.loc[games['year_of_release'].isnull(), 'platform'].map(platform_medians)

# Check for missing values in the 'year_of_release' column
print(f"Number of missing values in the 'year_of_release' column: {games['year_of_release'].isnull().sum()}")

Number of missing values in the 'year_of_release' column: 0


In [11]:
# convert 'year_of_release' to int
games['year_of_release'] = games['year_of_release'].astype(int)

A few of the games were able to tell me when they came out from the name but for the others i decided to group them by what platform they came out on and finding the median year for games that came out on that platform. Finally I converted it to int type because the variable only deals in whole numbers.

In [12]:
# display the rows with missing values in the 'genre' column
print(games[games['genre'].isnull()])

          name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    Unknown      GEN             1993   NaN      1.78      0.53      0.00   
14244  Unknown      GEN             1993   NaN      0.00      0.00      0.03   

       other_sales  critic_score user_score rating  
659           0.08           NaN        NaN    NaN  
14244         0.00           NaN        NaN    NaN  


In [13]:
# Filter the data for the year 1993
games_1993 = games[games['year_of_release'] == 1993]

# Find the most common genre
genre_counts = Counter(games_1993['genre'])
most_common_genre = genre_counts.most_common(1)[0][0]

# Fill the missing values in the 'genre' column with the most common genre
games['genre'] = games['genre'].fillna(most_common_genre)

# display the rows with missing values in the 'genre' column
print(games[games['genre'].isnull()])

Empty DataFrame
Columns: [name, platform, year_of_release, genre, na_sales, eu_sales, jp_sales, other_sales, critic_score, user_score, rating]
Index: []


Here I decided to find the most common genre for the year these games came out and apply it to these.

In [14]:
# check for duplicate rows
print(games.duplicated().sum())

0


In [15]:
# Add a new column 'total_sales' that calculates the sum of the individual sales columns
games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']

In [16]:
# Filter the games DataFrame to only include rows where the 'year_of_release' is 2016
games_2016 = games[games['year_of_release'] == 2016]

# Print the resulting DataFrame
games_2016

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
94,FIFA 17,PS4,2016,Sports,0.66,5.75,0.08,1.11,85.0,5,E,7.60
108,Pokemon Sun/Moon,3DS,2016,Role-Playing,2.98,1.45,2.26,0.45,NaN,NaN,NaN,7.14
171,Uncharted 4: A Thief's End,PS4,2016,Shooter,1.85,2.50,0.19,0.85,93.0,7.9,T,5.39
245,Call of Duty: Infinite Warfare,PS4,2016,Shooter,1.61,2.00,0.15,0.71,77.0,3.4,M,4.47
289,Battlefield 1,PS4,2016,Shooter,1.10,2.15,0.21,0.61,88.0,8.4,M,4.07
...,...,...,...,...,...,...,...,...,...,...,...,...
16699,The Longest 5 Minutes,PSV,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16703,Strawberry Nauts,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16710,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16712,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01


In [17]:
games_2016.info()

<class 'pandas.core.frame.DataFrame'>
Index: 502 entries, 94 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             502 non-null    object 
 1   platform         502 non-null    object 
 2   year_of_release  502 non-null    int32  
 3   genre            502 non-null    object 
 4   na_sales         502 non-null    float64
 5   eu_sales         502 non-null    float64
 6   jp_sales         502 non-null    float64
 7   other_sales      502 non-null    float64
 8   critic_score     232 non-null    float64
 9   user_score       296 non-null    object 
 10  rating           280 non-null    object 
 11  total_sales      502 non-null    float64
dtypes: float64(6), int32(1), object(5)
memory usage: 49.0+ KB


I need to fill the Null values int the critic score and user score still. I'll start by grouping by platform and genre and using the average critic score from each group.

In [18]:
# Group the data by 'platform' and 'genre', then calculate the mean 'critic_score' for each group
mean_critic_scores = games_2016.groupby(['platform', 'genre'])['critic_score'].mean().round(1).to_dict()

# Fill the missing values in the 'critic_score' column using the platform-genre mean critic scores
games_2016.loc[games_2016['critic_score'].isnull(), 'critic_score'] = games_2016.loc[games_2016['critic_score'].isnull(), ['platform', 'genre']].apply(lambda x: mean_critic_scores.get(tuple(x), games_2016['critic_score'].mean()), axis=1)

# Check for missing values in the 'critic_score' column
print(f"Number of missing values in the 'critic_score' column: {games_2016['critic_score'].isnull().sum()}")

# Get the rows with missing values in the 'critic_score' column
missing_critic_scores = games_2016[games_2016['critic_score'].isnull()]

# Print the missing rows
missing_critic_scores

Number of missing values in the 'critic_score' column: 65


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
2782,FIFA 17,PS3,2016,Sports,0.05,0.55,0.03,0.10,NaN,3.3,E,0.73
2973,Minecraft,WiiU,2016,Misc,0.28,0.17,0.18,0.04,NaN,NaN,NaN,0.67
5010,FIFA 17,X360,2016,Sports,0.07,0.28,0.00,0.03,NaN,1.7,E,0.38
5689,LEGO Marvel's Avengers,X360,2016,Action,0.18,0.11,0.00,0.03,NaN,5.7,E10+,0.32
6355,LEGO Marvel's Avengers,PS3,2016,Action,0.10,0.12,0.00,0.04,NaN,5.5,E10+,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...
16346,Lumo,PSV,2016,Puzzle,0.00,0.01,0.00,0.00,NaN,tbd,E10+,0.01
16558,Naruto Shippuden: Ultimate Ninja Storm Collection,PS3,2016,Action,0.00,0.01,0.00,0.00,NaN,NaN,NaN,0.01
16691,Dynasty Warriors: Eiketsuden,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16710,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01


For the remaining null values I'll fill them with the average score from the genre overall.

In [19]:
# Group the data by 'genre' and calculate the mean 'critic_score' for each genre
genre_mean_critic_scores = games_2016.groupby('genre')['critic_score'].mean().round(1).to_dict()

# Fill the remaining missing values in the 'critic_score' column with the genre average
games_2016.loc[games_2016['critic_score'].isnull(), 'critic_score'] = games_2016.loc[games_2016['critic_score'].isnull(), 'genre'].map(genre_mean_critic_scores)

# Check for remaining missing values in the 'critic_score' column
print(f"Number of missing values in the 'critic_score' column: {games_2016['critic_score'].isnull().sum()}")

# Get the rows with missing values in the 'critic_score' column
missing_critic_scores = games_2016[games_2016['critic_score'].isnull()]

# Print the missing rows
missing_critic_scores

Number of missing values in the 'critic_score' column: 1


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
16346,Lumo,PSV,2016,Puzzle,0.0,0.01,0.0,0.0,NaN,tbd,E10+,0.01


For the final one I'll fill it with the average for all games.

In [20]:
# Calculate the overall mean 'critic_score' for all games
overall_mean_critic_score = games_2016['critic_score'].mean().round(1)

# Fill the remaining missing value in the 'critic_score' column with the overall mean
games_2016.loc[games_2016['critic_score'].isnull(), 'critic_score'] = overall_mean_critic_score

# Check for remaining missing values in the 'critic_score' column
print(f"Number of missing values in the 'critic_score' column: {games_2016['critic_score'].isnull().sum()}")

Number of missing values in the 'critic_score' column: 0


For user scores with a tbd value I will give them 5.0 since thats the average of potential scores. Then I'll convert to float so that i can easily work with the numbers. 

In [31]:
# Replace 'tbd' values in the 'user_score' column with 5.0
games_2016.loc[:, 'user_score'] = games_2016['user_score'].replace('tbd', 5.0)

# Convert the 'user_score' column to float
games_2016.loc[:, 'user_score'] = games_2016['user_score'].astype(float)

# Group the data by 'genre' and 'platform', calculate the mean 'user_score' for each group
group_mean_user_scores = games_2016.groupby(['genre', 'platform'])['user_score'].mean().round(1).to_dict()

# Fill the remaining missing values in the 'user_score' column with the group average
games_2016.loc[games_2016['user_score'].isnull(), 'user_score'] = games_2016.loc[games_2016['user_score'].isnull(), ['genre', 'platform']].apply(lambda x: group_mean_user_scores[(x['genre'], x['platform'])], axis=1)

# Check for remaining missing values in the 'user_score' column
print(f"Number of missing values in the 'user_score' column: {games_2016['user_score'].isnull().sum()}")

Number of missing values in the 'user_score' column: 0


For the remaining null values I'll fill them with the average score from the genres overall.

In [22]:
# Group the data by 'genre' and calculate the mean 'user_score' for each genre
genre_mean_user_scores = games_2016.groupby('genre')['user_score'].mean().round(1).to_dict()

# Fill the remaining missing values in the 'user_score' column with the genre average
games_2016.loc[games_2016['user_score'].isnull(), 'user_score'] = games_2016.loc[games_2016['user_score'].isnull(), 'genre'].map(genre_mean_user_scores)

# Check for remaining missing values in the 'user_score' column
print(f"Number of missing values in the 'user_score' column: {games_2016['user_score'].isnull().sum()}")

Number of missing values in the 'user_score' column: 0


In [23]:
games_2016.info()

<class 'pandas.core.frame.DataFrame'>
Index: 502 entries, 94 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             502 non-null    object 
 1   platform         502 non-null    object 
 2   year_of_release  502 non-null    int32  
 3   genre            502 non-null    object 
 4   na_sales         502 non-null    float64
 5   eu_sales         502 non-null    float64
 6   jp_sales         502 non-null    float64
 7   other_sales      502 non-null    float64
 8   critic_score     502 non-null    float64
 9   user_score       502 non-null    object 
 10  rating           280 non-null    object 
 11  total_sales      502 non-null    float64
dtypes: float64(6), int32(1), object(5)
memory usage: 49.0+ KB


Finally I'll fill the null values in the ratings column

In [24]:
games_2016['rating'].value_counts(dropna=False)

rating
NaN     222
T        86
M        78
E        66
E10+     50
Name: count, dtype: int64

I'll group these by Genre and Platform again but this time find the mode of the ratings for each

In [25]:
# Drop null values from data frame before finding the modes
most_common_ratings = games_2016.dropna(subset=['rating'])

# Group the DataFrame by 'genre' and 'platform', and find the most common 'rating' for each group
most_common_ratings = most_common_ratings.groupby(['genre', 'platform'])['rating'].apply(lambda x: x.mode()[0])

# Create a dictionary to map the group keys to the most common ratings
rating_map = most_common_ratings.to_dict()

# Fill the null values in the 'rating' column with the most common rating for each group
games_2016.loc[:, 'rating'] = games_2016.apply(lambda row: rating_map.get((row['genre'], row['platform']), row['rating']), axis=1)

games_2016['rating'].value_counts(dropna=False)

rating
M       205
T       110
E       109
E10+     69
NaN       9
Name: count, dtype: int64

In [28]:
# Find the most common rating for each genre
most_common_ratings = games_2016.groupby('genre')['rating'].apply(lambda x: x.mode().iloc[0])

# Create a dictionary to map the genres to the most common ratings
rating_map = most_common_ratings.to_dict()

# Fill the null values in the 'rating' column with the most common rating for each genre
games_2016.loc[:, 'rating'] = games_2016['rating'].fillna(games_2016['genre'].map(rating_map))

games_2016['rating'].value_counts(dropna=False)

rating
M       208
T       113
E       112
E10+     69
Name: count, dtype: int64

In [29]:
games_2016.info()

<class 'pandas.core.frame.DataFrame'>
Index: 502 entries, 94 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             502 non-null    object 
 1   platform         502 non-null    object 
 2   year_of_release  502 non-null    int32  
 3   genre            502 non-null    object 
 4   na_sales         502 non-null    float64
 5   eu_sales         502 non-null    float64
 6   jp_sales         502 non-null    float64
 7   other_sales      502 non-null    float64
 8   critic_score     502 non-null    float64
 9   user_score       502 non-null    object 
 10  rating           502 non-null    object 
 11  total_sales      502 non-null    float64
dtypes: float64(6), int32(1), object(5)
memory usage: 49.0+ KB
